In [0]:
from google.colab import auth
auth.authenticate_user()

# from google.colab import drive
# drive.mount('/gdrive')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
import os

# @title Install Dependences
!pip install -q keras-bert

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

In [0]:
# https://www.kaggle.com/taindow/bert-a-fine-tuning-example
import pprint
import time
import json
import sys
import collections
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
from tqdm import *


print("Tensorflow version " + tf.__version__)

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import run_classifier
import modeling
import optimization
import tokenization

TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

Tensorflow version 1.13.1
Cloning into 'bert_repo'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 232.46 KiB | 3.10 MiB/s, done.
Resolving deltas: 100% (186/186), done.
TPU address is grpc://10.59.189.82:8470


In [0]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
    from google.colab import auth
    auth.authenticate_user() # Authenticates the backend and also the TPU using your credentials so that they can access your private GCS buckets
    
    with tf.Session(TPU_ADDRESS) as session:
        print('TPU devices:')
        pprint.pprint(session.list_devices())

        # Upload credentials to TPU.
        with open('/content/adc.json', 'r') as f:
            auth_info = json.load(f)
            tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
            # Now credentials are set for all future sessions on this TPU.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 16911102095813016144),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9032940763894176605),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9397770395000325313),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2799334155243662634),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9647770266160103276),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4093256256244460234),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9682279001468089904),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 1376157206058696047),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 13851991804207779683),
 _DeviceAttributes(/job:tpu_work

In [0]:
TASK = 'CoLA' #@param {type:"string"}
assert TASK in ('MRPC', 'CoLA'), 'Only (MRPC, CoLA) are demonstrated here.'

BUCKET = 'tj-kaggle/toxic' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12    : uncased BERT base model
#   uncased_L-24_H-1024_A-16   : uncased BERT large model
#   cased_L-12_H-768_A-12      : cased BERT base model
#   cased_L-24_H-1024_A-16     : uncased BERT large model

BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}


# Setup task specific model and TPU running config.
BERT_PRETRAINED_DIR = "gs://{}/model/{}/".format(BUCKET, BERT_MODEL)
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls {BERT_PRETRAINED_DIR}

CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
BERT_CONFIG = modeling.BertConfig.from_json_file(CONFIG_FILE)
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')

checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')


IS_CASED_MODEL = False
BERT_STR = "uncased"

if BERT_MODEL[:5] == "cased" :
    IS_CASED_MODEL = True
    BERT_STR = "cased"

print("BERT_CASED : {}".format(BERT_STR))

# tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer = run_classifier.tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=IS_CASED_MODEL)
print(tokenizer.tokenize("This here's an example of using the BERT tokenizer"))

print("Check the capital of word 'bert'..")

***** BERT pretrained directory: gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/ *****
gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/bert_config.json
gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/vocab.txt
BERT_CASED : uncased
['[UNK]', 'here', "'", 's', 'an', 'example', 'of', 'using', 'the', '[UNK]', 'token', '##izer']
Check the capital of word 'bert'..


In [0]:
!gsutil -m cp {BERT_PRETRAINED_DIR}* /content/

Copying gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/bert_config.json...
Copying gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/bert_model.ckpt.index...
Copying gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001...
Copying gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/bert_model.ckpt.meta...
Copying gs://tj-kaggle/toxic/model/uncased_L-12_H-768_A-12/vocab.txt...
\ [5/5 files][421.1 MiB/421.1 MiB] 100% Done                                    
Operation completed over 5 objects/421.1 MiB.                                    


In [0]:
DATA_PATH = '/content/'

In [0]:
CONFIG_FILE = os.path.join(DATA_PATH, 'bert_config.json')
VOCAB_FILE = os.path.join(DATA_PATH, 'vocab.txt')
checkpoint_file = os.path.join(DATA_PATH, 'bert_model.ckpt')

In [0]:
from keras_bert import load_trained_model_from_checkpoint

In [0]:
# @title Constants

SEQ_LEN = 300
BATCH_SIZE = 128
EPOCHS = 2
LR = 2e-5
loss_weight = 3.2092275837114372

In [0]:
base_model = load_trained_model_from_checkpoint(CONFIG_FILE, checkpoint_file, training=True, trainable=True, seq_len=SEQ_LEN)
base_model.summary(line_length=120)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               (None, 300)                0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             (None, 300)                0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 300, 768), (30522, 23440896      Input-T

# Generator

In [0]:
! gsutil -m cp gs://tj-kaggle/toxic/data/X_seq_300.npy gs://tj-kaggle/toxic/data/y_aux* gs://tj-kaggle/toxic/data/y.npy  /content/

Copying gs://tj-kaggle/toxic/data/X_seq_300.npy...
Copying gs://tj-kaggle/toxic/data/y_aux_train.csv...
Copying gs://tj-kaggle/toxic/data/y.npy...
| [3/3 files][  4.1 GiB/  4.1 GiB] 100% Done  25.8 MiB/s ETA 00:00:00           
Operation completed over 3 objects/4.1 GiB.                                      


In [0]:
# ! gsutil -m cp gs://tj-kaggle/toxic/keras_bert_2ep.h5  /content/

In [0]:
import numpy as np
X = np.load('X_seq_300.npy')
y = np.load('y.npy')

import pandas as pd 
y_aux_train = pd.read_csv('y_aux_train.csv')

In [0]:
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, data, labels, aux_labels, batch_size=128, x_dim=(220), y_dim=(2), y_aux_dim=(6), shuffle=True):
        'Initialization'
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.y_aux_dim = y_aux_dim
        self.batch_size = batch_size
        self.labels = labels
        self.aux_labels = aux_labels
        self.data = data
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, X_seg, X_mask, y, y_aux = self.__data_generation(indexes)

        return [X, X_seg, X_mask], [y, y_aux]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, self.x_dim))
        y = np.empty((self.batch_size, self.y_dim), dtype=int)
        
        X = np.array([self.data[i] for i in indexes])
        X_seg = np.zeros((self.batch_size, self.x_dim))
        X_mask = np.ones((self.batch_size, self.x_dim))
        
        y = np.array([self.labels[i] for i in indexes])
        y_aux = np.array([self.aux_labels.loc[i,:] for i in indexes])


        return X, X_seg, X_mask, y, y_aux 

In [0]:
def myGenerator(X, y, y_aux_train, batch_size, max_seq_len):
    x_dim = (max_seq_len)
    y_dim = (2)
    y_aux_dim = (6)
    
    # init
#     X = np.empty((batch_size, x_dim), dtype=np.int)
#     y = np.empty((batch_size, y_dim), dtype=np.float)
    data_size = len(X)
    remainder = data_size % batch_size
    indexes = np.arange(data_size)
    repeat = int(np.ceil(data_size / batch_size))
    
    # loop
    for index in range(0, repeat):
        i_batch = indexes[index*batch_size : (index+1)*batch_size]
        X_batch = np.array([X[i] for i in i_batch], dtype=np.int)
        y_batch = np.array([y[i] for i in i_batch], dtype=np.float)     

        y_aux_batch = np.array([y_aux_train.loc[i,:] for i in i_batch], dtype=np.float)
        
        if index == repeat -1 : # last
            X_seg_batch = np.zeros((remainder, x_dim))
            X_mask_batch = np.ones((remainder, x_dim))
            
        else :
            X_seg_batch = np.zeros((batch_size, x_dim))
            X_mask_batch = np.ones((batch_size, x_dim))

        yield [X_batch, X_seg_batch, X_mask_batch], [y_batch, y_aux_batch]

In [0]:
y_aux_train = y_aux_train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]

In [0]:
params = {'batch_size' : BATCH_SIZE, 
          'x_dim' :  (SEQ_LEN),
          'y_dim' : (2),
          'y_aux_dim' : (6),
          'shuffle' : True}
training_generator = DataGenerator(X, y, y_aux_train, **params)

In [0]:
# training_generator = myGenerator(X, y, y_aux_train, BATCH_SIZE, SEQ_LEN)

In [0]:
from tensorflow.python.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

def custom_loss(y_true, y_pred):
    return binary_crossentropy(K.reshape(y_true[:,0],(-1,1)), y_pred) * y_true[:,1]

In [0]:
base_model.get_layer('NSP-Dense').output

<tf.Tensor 'NSP-Dense/Tanh:0' shape=(?, 768) dtype=float32>

In [0]:
# from tensorflow.python import keras
from tensorflow.python import keras
from keras_bert import calc_train_steps

extract = base_model.get_layer('Extract').output
# target_layer = keras.layers.Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02), name='target_layer')(extract)
# aux_layer = keras.layers.Dense(6, activation='sigmoid', kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),  name='aux_layer')(extract)

final_layer = keras.layers.Dense(256, activation='relu')(extract)
final_layer = keras.layers.Dropout(0.25)(final_layer)

target_layer = keras.layers.Dense(1, activation='sigmoid', name='target_layer')(final_layer)
aux_layer = keras.layers.Dense(6, activation='sigmoid', name='aux_layer')(final_layer)

# decay_steps, warmup_steps = calc_train_steps(
#     y.shape[0],
#     batch_size=BATCH_SIZE,
#     epochs=EPOCHS,
# )


model = keras.models.Model(inputs=base_model.input, outputs=[target_layer, aux_layer])
# model.compile(
#     optimizer = AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
#     loss = [custom_loss, 'binary_crossentropy'],
#     loss_weights = [loss_weight, 1.0]
# )
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 300)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 300)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 300, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 300, 768)     1536        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

In [0]:
# class AdamWarmup(tf.train.AdamOptimizer):
#     def __init__(self, decay_steps, warmup_steps, min_lr=0.0,
#                  lr=0.001, beta_1=0.9, beta_2=0.999,
#                  epsilon=None, kernel_weight_decay=0., bias_weight_decay=0.,
#                  amsgrad=False, **kwargs):
#         super(AdamWarmup, self).__init__(**kwargs)
#         with K.name_scope(self.__class__.__name__):
#             self.decay_steps = K.variable(decay_steps, name='decay_steps')
#             self.warmup_steps = K.variable(warmup_steps, name='warmup_steps')
#             self.min_lr = K.variable(min_lr, name='min_lr')
#             self.iterations = K.variable(0, dtype='int64', name='iterations')
#             self.lr = K.variable(lr, name='lr')
#             self.beta_1 = K.variable(beta_1, name='beta_1')
#             self.beta_2 = K.variable(beta_2, name='beta_2')
#             self.kernel_weight_decay = K.variable(kernel_weight_decay, name='kernel_weight_decay')
#             self.bias_weight_decay = K.variable(bias_weight_decay, name='bias_weight_decay')
#         if epsilon is None:
#             epsilon = K.epsilon()
#         self.epsilon = epsilon
#         self.initial_kernel_weight_decay = kernel_weight_decay
#         self.initial_bias_weight_decay = bias_weight_decay
#         self.amsgrad = amsgrad

#     def get_updates(self, loss, params):
#         grads = self.get_gradients(loss, params)
#         self.updates = [K.update_add(self.iterations, 1)]

#         t = K.cast(self.iterations, K.floatx()) + 1

#         lr = K.switch(
#             t <= self.warmup_steps,
#             self.lr * (t / self.warmup_steps),
#             self.lr * (1.0 - K.minimum(t, self.decay_steps) / self.decay_steps),
#         )

#         lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
#                      (1. - K.pow(self.beta_1, t)))

#         ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
#         vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
#         if self.amsgrad:
#             vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
#         else:
#             vhats = [K.zeros(1) for _ in params]
#         self.weights = [self.iterations] + ms + vs + vhats

#         for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
#             m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
#             v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
#             if self.amsgrad:
#                 vhat_t = K.maximum(vhat, v_t)
#                 p_t = m_t / (K.sqrt(vhat_t) + self.epsilon)
#                 self.updates.append(K.update(vhat, vhat_t))
#             else:
#                 p_t = m_t / (K.sqrt(v_t) + self.epsilon)

#             if 'bias' in p.name:
#                 if self.initial_bias_weight_decay > 0.0:
#                     p_t += self.bias_weight_decay * p
#             else:
#                 if self.initial_kernel_weight_decay > 0.0:
#                     p_t += self.kernel_weight_decay * p
#             p_t = p - lr_t * p_t

#             self.updates.append(K.update(m, m_t))
#             self.updates.append(K.update(v, v_t))
#             new_p = p_t

#             if getattr(p, 'constraint', None) is not None:
#                 new_p = p.constraint(new_p)

#             self.updates.append(K.update(p, new_p))
#         return self.updates

#     def get_config(self):
#         config = {
#             'decay_steps': float(K.get_value(self.decay_steps)),
#             'warmup_steps': float(K.get_value(self.warmup_steps)),
#             'min_lr': float(K.get_value(self.min_lr)),
#             'lr': float(K.get_value(self.lr)),
#             'beta_1': float(K.get_value(self.beta_1)),
#             'beta_2': float(K.get_value(self.beta_2)),
#             'epsilon': self.epsilon,
#             'kernel_weight_decay': float(K.get_value(self.kernel_weight_decay)),
#             'bias_weight_decay': float(K.get_value(self.bias_weight_decay)),
#             'amsgrad': self.amsgrad,
#         }
#         base_config = super(AdamWarmup, self).get_config()
#         return dict(list(base_config.items()) + list(config.items()))

In [0]:
# @title Initialize Variables
import tensorflow as tf
import tensorflow.keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [0]:
import tensorflow as tf
from keras_bert import get_custom_objects

tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
)

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
    tpu_model.compile(
#         optimizer = AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
        optimizer = tf.train.AdamOptimizer(learning_rate=LR),
        loss = [custom_loss, 'binary_crossentropy'],
        loss_weights = [loss_weight, 1.0]
    )

INFO:tensorflow:Querying Tensorflow master (grpc://10.59.189.82:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 16911102095813016144)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9032940763894176605)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9397770395000325313)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2799334155243662634)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9647770266160103276)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/

In [0]:
filepath = "keras-bert-datagen-{epoch:02d}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, save_weights_only=False)
callbacks_list = [checkpoint]

In [0]:
int(np.ceil(X.shape[0] / BATCH_SIZE)) - 1

14101

In [0]:
with tf.keras.utils.custom_object_scope(get_custom_objects()):
    tpu_model.fit_generator(
        training_generator, 
        epochs=2,
        callbacks=callbacks_list
    )

Epoch 1/2
14099/14100 [============================>.] - ETA: 0s - loss: 0.2901 - target_layer_loss: 0.0784 - aux_layer_loss: 0.0784
Epoch 00001: saving model to keras-bert-datagen-01.h5
INFO:tensorflow:Copying TPU weights to the CPU
14100/14100 [==============================] - 9715s 689ms/step - loss: 0.2901 - target_layer_loss: 0.0784 - aux_layer_loss: 0.0784
Epoch 2/2
 5884/14100 [===========>..................] - ETA: 1:34:32 - loss: 0.2739 - target_layer_loss: 0.0751 - aux_layer_loss: 0.0751

In [0]:
!gsutil -m cp /content/*datagen* gs://tj-kaggle/toxic/

In [0]:
fileName = '/content/bert_add_dense_adam_preprocessed_seq300.h5'
tpu_model.save_weights(fileName)

INFO:tensorflow:Copying TPU weights to the CPU


In [0]:
!gsutil -m cp {fileName} gs://tj-kaggle/toxic/

Copying file:///content/bert_add_dense_adam_preprocessed_seq300.h5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/415.8 MiB.                                    


In [0]:
# with tf.keras.utils.custom_object_scope(get_custom_objects()):
#     tpu_model.fit(
#         [X, X_seg, X_mask],
#         [y, y_aux_train.values],
#         epochs=EPOCHS,
#         batch_size=BATCH_SIZE,
#     )

# Predfict , make submission

In [0]:
! gsutil -m cp gs://tj-kaggle/toxic/data/X_test.npy gs://tj-kaggle/toxic/data/*submission* /content/
        
X_test = np.load('X_test.npy')
X_seg_input = np.zeros((X_test.shape[0], SEQ_LEN))
X_mask_input = np.ones((X_test.shape[0], SEQ_LEN))
with tf.keras.utils.custom_object_scope(get_custom_objects()):
    history = tpu_model.predict([X_test[:32], X_seg_input[:32], X_mask_input[:32]], batch_size=32, verbose=1)
submission = pd.read_csv('sample_submission.csv', index_col='id')[:32]
submission['prediction'] = history[0]
submission.to_csv('submission.csv')

print(submission.head(20))

INFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(16, 300), dtype=tf.float32, name='Input-Token_20'), TensorSpec(shape=(16, 300), dtype=tf.float32, name='Input-Segment_20'), TensorSpec(shape=(16, 300), dtype=tf.float32, name='Input-Masked_20')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input-Token
INFO:tensorflow:Remapping placeholder for Input-Segment
INFO:tensorflow:Remapping placeholder for Input-Masked
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 45.296220779418945 secs
97280/97320 [============================>.] - ETA: 0sINFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(5, 300), dtype=tf.float32, name='Input-Token_20'), TensorSpec(shape=(5, 300), dtype=tf.float32, name='Input-Segment_20'), TensorSpec(shape=(5, 300), dtype=tf.float32, name='Input-Masked_20')]
INFO:tensorflow:Overriding default placeholder

## if you want to load checkpoint..

In [0]:
model.load_weights('/content/keras-bert-datagen-01.h5')
model.compile(loss=[custom_loss,'binary_crossentropy'], loss_weights=[loss_weight, 1.0], optimizer=tf.train.AdamOptimizer(learning_rate=LR))

In [0]:
history = model.predict([X_test[:20], X_seg_input[:20], X_mask_input[:20]], batch_size = 20, verbose=1)
submission = pd.read_csv('sample_submission.csv', index_col='id')[:20]

submission['prediction'] = history[0]
submission.to_csv('submission.csv')

print(submission.head(20))

20/20 [==============================] - 2s 98ms/sample
         prediction
id                 
7000000    0.002732
7000001    0.001375
7000002    0.004579
7000003    0.002157
7000004    0.947316
7000005    0.001487
7000006    0.003871
7000007    0.003015
7000008    0.001783
7000009    0.002385
7000010    0.002080
7000011    0.417077
7000012    0.003129
7000013    0.001860
7000014    0.002776
7000015    0.001485
7000016    0.285142
7000017    0.002427
7000018    0.347868
7000019    0.019464


In [0]:
tpu_model.save_weights('/content/keras_bert_ep{}.h5'.format(4))

INFO:tensorflow:Copying TPU weights to the CPU


In [0]:
!gsutil -m cp /content/keras-bert*.h5 gs://tj-kaggle/toxic/